In [2]:
!nvidia-smi

Sun Aug  4 00:04:08 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.90.07              Driver Version: 550.90.07      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   58C    P8             10W /   70W |       1MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [3]:
%pip install transformers==4.32 \
             tensorflow \
             datasets \
             loralib \
             peft -q

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
kaggle-environments 1.14.15 requires transformers>=4.33.1, but you have transformers 4.32.0 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [4]:
%pip install scikit-learn -q

Note: you may need to restart the kernel to use updated packages.


In [5]:
from transformers import AutoModelForSeq2SeqLM
model_name = "facebook/nllb-200-distilled-600M"
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/846 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

In [6]:
from transformers import NllbTokenizer
s_lang="hin_Deva"
t_lang="kan_Deva"
tokenizer = NllbTokenizer.from_pretrained('facebook/nllb-200-distilled-600M', src_lang=s_lang, tgt_lang=t_lang)

sentencepiece.bpe.model:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/3.55k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/564 [00:00<?, ?B/s]

## **Loading and preparing the Dataset**

In [7]:
h_path= '/kaggle/input/ft-mt-llm-data/Parallel_data_Hi.txt'
k_path= '/kaggle/input/ft-mt-llm-data/Parallel_data_KP.txt'

def read_and_filter(file_path):
    sentences= []
    with open(file_path, 'r', encoding= 'utf-8') as file:
        for line in file:
            stripped_line= line.strip()
            if stripped_line:
                sentences.append(line)
                
    return sentences

h_sen= read_and_filter(h_path)
k_sen= read_and_filter(k_path)

if len(h_sen) != len(k_sen):
    raise ValueError("The number of sentences do not match!")
else:
    print("The number of sentences match!")

The number of sentences match!


In [8]:
print(f"Total number of sentences: {len(h_sen)}")

Total number of sentences: 20307


In [9]:
for i in range(5):
    print(f"Hindi: {h_sen[i]}")
    print(f"Kangri: {k_sen[i]}")
    print("--"*40)

Hindi: हिन्दी-हिमाचली पहाड़ी भाषा का नाम सुनकर ही कुछ लोग विदक जाते हैं।

Kangri: हिन्दी-हिमाचली पहाडी काद रोह नाऔ शूनिंदैए दौक मानुष सौर्मींदैसा।

--------------------------------------------------------------------------------
Hindi: अगर इस मुद्दे पर दूसरे प्रदेशों के निवासियों को कोई तकलीफ हो तो भी समझ आती है ।

Kangri: कियै हौयौ बातै कौरि ओरिक प्रदेशाए मानुषौलै किछ़ तकलीफ हुँदेन ताबिह सौमौज़ आछ़दौँस।

--------------------------------------------------------------------------------
Hindi: किंतु अपने ही प्रदेश की भाषा के प्रति लोगों की के मन की संकीर्णता समझ नहीं आती ।

Kangri: पर आपनैए प्रदेशाई काद रिह ताईंयै मानुषौ रोह मौनै कैह मारे बौरताव सौमौज़ नाछ़दौस।

--------------------------------------------------------------------------------
Hindi: पता नहीं ऐसा कर के क्या हासिल होगा ।

Kangri: ज़ानि हैयौ कौरिंदै कयौ पौरेस।

--------------------------------------------------------------------------------
Hindi: अगर हम अपनी माता (हिमाचली भाषा) के नहीं हो सकते तो विमाता के कहाँ होगे  ।

Kan

In [10]:
dataset= list(zip(h_sen, k_sen))

import csv
import pandas as pd

file_name= "hi-ka-translations.csv"

with open(file_name, 'w', encoding= 'utf-8') as file:
    writer= csv.writer(file)
    writer.writerow(['Hindi', 'Kangri'])
    
    for Hindi, Kangri in dataset:
        writer.writerow([Hindi, Kangri])

In [11]:
dataframe= pd.read_csv("/kaggle/working/hi-ka-translations.csv")
dataframe

,Hindi,Kangri
0,हिन्दी-हिमाचली पहाड़ी भाषा का नाम सुनकर ही कुछ...,हिन्दी-हिमाचली पहाडी काद रोह नाऔ शूनिंदैए दौक ...
1,अगर इस मुद्दे पर दूसरे प्रदेशों के निवासियों क...,कियै हौयौ बातै कौरि ओरिक प्रदेशाए मानुषौलै किछ...
2,किंतु अपने ही प्रदेश की भाषा के प्रति लोगों की...,पर आपनैए प्रदेशाई काद रिह ताईंयै मानुषौ रोह मौ...
3,पता नहीं ऐसा कर के क्या हासिल होगा ।\n,ज़ानि हैयौ कौरिंदै कयौ पौरेस।\n
4,अगर हम अपनी माता (हिमाचली भाषा) के नहीं हो सकत...,कियै तामोरि आपनिये यो (हिमाचली बोली) रै निफिरि...
...,...,...
20302,जनाब ₹3 तो कंडक्टर को दे दिए पर 20 पैसे खुले ह...,जनाब त्रोन रुपया तह कंडक्टर लाह देई देह तेबिह ...
20303,पहले तो वह अस्पताल के लिए तैयार नहीं हुआ पर मु...,पोईले ताह सोह अस्‍पताल रोह ताई तैयार निफरिह ते...
20304,उसने मुट्ठी में गिन कर रखे हुए पैसे मेरे सामने...,तिनिये मुठोह केह गोनाई जागुन रुपया लाह मेरोह स...
20305,कई दिन गुजर गए मेरी चिट्ठी का भोलू राम पर क्य...,मेरे पास कोई आया गया ही नहीं यूनिवर्सिटी के इम...


> **Repeating to get more data:**

In [12]:
repeat_times = 3
augmented_dataframe = pd.concat([dataframe] * repeat_times)
augmented_dataframe = augmented_dataframe.reset_index(drop=True)
augmented_dataframe.to_csv("/kaggle/working/augmented_hi-ka-translations.csv")
augmented_dataframe

,Hindi,Kangri
0,हिन्दी-हिमाचली पहाड़ी भाषा का नाम सुनकर ही कुछ...,हिन्दी-हिमाचली पहाडी काद रोह नाऔ शूनिंदैए दौक ...
1,अगर इस मुद्दे पर दूसरे प्रदेशों के निवासियों क...,कियै हौयौ बातै कौरि ओरिक प्रदेशाए मानुषौलै किछ...
2,किंतु अपने ही प्रदेश की भाषा के प्रति लोगों की...,पर आपनैए प्रदेशाई काद रिह ताईंयै मानुषौ रोह मौ...
3,पता नहीं ऐसा कर के क्या हासिल होगा ।\n,ज़ानि हैयौ कौरिंदै कयौ पौरेस।\n
4,अगर हम अपनी माता (हिमाचली भाषा) के नहीं हो सकत...,कियै तामोरि आपनिये यो (हिमाचली बोली) रै निफिरि...
...,...,...
60916,जनाब ₹3 तो कंडक्टर को दे दिए पर 20 पैसे खुले ह...,जनाब त्रोन रुपया तह कंडक्टर लाह देई देह तेबिह ...
60917,पहले तो वह अस्पताल के लिए तैयार नहीं हुआ पर मु...,पोईले ताह सोह अस्‍पताल रोह ताई तैयार निफरिह ते...
60918,उसने मुट्ठी में गिन कर रखे हुए पैसे मेरे सामने...,तिनिये मुठोह केह गोनाई जागुन रुपया लाह मेरोह स...
60919,कई दिन गुजर गए मेरी चिट्ठी का भोलू राम पर क्य...,मेरे पास कोई आया गया ही नहीं यूनिवर्सिटी के इम...


In [13]:
from sklearn.model_selection import train_test_split
train_val_df, test_dataset= train_test_split(augmented_dataframe, test_size=0.1, random_state=40)
train_dataset, evaluation_dataset= train_test_split(train_val_df, test_size=0.1, random_state=40)
print('Training dataset shape: ', train_dataset.shape)
print('Validation dataset shape: ', evaluation_dataset.shape)
print('Testing dataset shape: ', test_dataset.shape)

Training dataset shape:  (49345, 2)
Validation dataset shape:  (5483, 2)
Testing dataset shape:  (6093, 2)


> **Converting pandas dataframe to pyarrow**

In [14]:
from datasets import Dataset

train_dataset= Dataset.from_pandas(train_dataset)
val_dataset= Dataset.from_pandas(evaluation_dataset)
test_dataset= Dataset.from_pandas(test_dataset)

train_dataset= train_dataset.remove_columns("__index_level_0__")
val_dataset= val_dataset.remove_columns("__index_level_0__")
test_dataset= test_dataset.remove_columns("__index_level_0__")

In [15]:
import datasets
main_dataset= datasets.DatasetDict({
    'train': train_dataset,
    'validation': val_dataset,
    'test': test_dataset
})

In [16]:
main_dataset

DatasetDict({
    train: Dataset({
        features: ['Hindi', 'Kangri'],
        num_rows: 49345
    })
    validation: Dataset({
        features: ['Hindi', 'Kangri'],
        num_rows: 5483
    })
    test: Dataset({
        features: ['Hindi', 'Kangri'],
        num_rows: 6093
    })
})

## **Tokenizing and getting ready for Training**

> **Checking the number of unknown tokens(UNK) in the dataset:**

In [17]:
from tqdm.auto import tqdm, trange
import random

texts_with_unk = [
    text for text in tqdm(dataframe.Hindi) 
    if tokenizer.unk_token_id in tokenizer(text).input_ids
]
print(len(texts_with_unk))

  0%|          | 0/20307 [00:00<?, ?it/s]

156


In [18]:
texts_with_unk = [
    text for text in tqdm(dataframe.Kangri) 
    if tokenizer.unk_token_id in tokenizer(text).input_ids
]
print(len(texts_with_unk))

  0%|          | 0/20307 [00:00<?, ?it/s]

364


The number of unknown tokens is fairly low therefore, putting off expanding the tokenizer vocabulary. 

>  **Adding a new language code to the tokenizer:**

In [19]:
def fix_tokenizer(tokenizer, new_lang='kan_Deva'):

    old_len = len(tokenizer) - int(new_lang in tokenizer.added_tokens_encoder)
    tokenizer.lang_code_to_id[new_lang] = old_len-1
    tokenizer.id_to_lang_code[old_len-1] = new_lang

    tokenizer.fairseq_tokens_to_ids["<mask>"] = len(tokenizer.sp_model) + len(tokenizer.lang_code_to_id) + tokenizer.fairseq_offset

    tokenizer.fairseq_tokens_to_ids.update(tokenizer.lang_code_to_id)
    tokenizer.fairseq_ids_to_tokens = {v: k for k, v in tokenizer.fairseq_tokens_to_ids.items()}
    if new_lang not in tokenizer._additional_special_tokens:
        tokenizer._additional_special_tokens.append(new_lang)
    tokenizer.added_tokens_encoder = {}
    tokenizer.added_tokens_decoder = {}
    
    return tokenizer

tokenizer= fix_tokenizer(tokenizer)

In [20]:
model.resize_token_embeddings(len(tokenizer))

You are resizing the embedding layer without providing a `pad_to_multiple_of` parameter. This means that the new embeding dimension will be 256205. This might induce some performance reduction as *Tensor Cores* will not be available. For more details  about this, or help on choosing the correct value for resizing, refer to this guide: https://docs.nvidia.com/deeplearning/performance/dl-performance-matrix-multiplication/index.html#requirements-tc


Embedding(256205, 1024)

In [21]:
added_token_id = tokenizer.convert_tokens_to_ids('kan_Deva')
similar_lang_id = tokenizer.convert_tokens_to_ids('hin_Deva')
embeds = model.model.shared.weight.data
# moving the embedding for "mask" to its new position
embeds[added_token_id+1] =embeds[added_token_id]
# initializing new language token with a token of a similar language
embeds[added_token_id] = embeds[similar_lang_id]

> **Tokenizing the main_dataset**

In [22]:
max_length= 128

def preprocess_function(examples):
    inputs= [ex for ex in examples["Hindi"]]
    targets= [ex for ex in examples["Kangri"]]
    model_inputs = tokenizer(
        inputs, text_target=targets, max_length=max_length, truncation=True
    )
    return model_inputs

In [23]:
tokenized_datasets = main_dataset.map(
    preprocess_function,
    batched=True,
)

Map:   0%|          | 0/49345 [00:00<?, ? examples/s]

Map:   0%|          | 0/5483 [00:00<?, ? examples/s]

Map:   0%|          | 0/6093 [00:00<?, ? examples/s]

In [24]:
tokenized_datasets= tokenized_datasets.remove_columns(['Hindi', 'Kangri'])

In [25]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 49345
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 5483
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 6093
    })
})

> **Data Collation**

In [26]:
%pip install torch -q

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


Note: you may need to restart the kernel to use updated packages.


In [27]:
from transformers import DataCollatorForSeq2Seq
#generation_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="pt", pad_to_multiple_of=128)

#Pytorch
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

2024-08-04 00:06:44.086188: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-04 00:06:44.086347: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-04 00:06:44.218629: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [28]:
"""
# Since nllb doesn't support tensorlfow therefore going with pytorch
tf_train_dataset = tokenized_datasets["train"].to_tf_dataset(
    columns=["input_ids","attention_mask"],
    label_cols=["labels"],
    shuffle=True,
    collate_fn=generation_data_collator,
    batch_size=8,
)

tf_validation_dataset = tokenized_datasets["validation"].to_tf_dataset(
    columns=["input_ids","attention_mask"],
    label_cols=["labels"],
    shuffle=False,
    collate_fn=data_collator,
    batch_size=8,
)
"""

'\n# Since nllb doesn\'t support tensorlfow therefore going with pytorch\ntf_train_dataset = tokenized_datasets["train"].to_tf_dataset(\n    columns=["input_ids","attention_mask"],\n    label_cols=["labels"],\n    shuffle=True,\n    collate_fn=generation_data_collator,\n    batch_size=8,\n)\n\ntf_validation_dataset = tokenized_datasets["validation"].to_tf_dataset(\n    columns=["input_ids","attention_mask"],\n    label_cols=["labels"],\n    shuffle=False,\n    collate_fn=data_collator,\n    batch_size=8,\n)\n'

In [29]:
from torch.utils.data import DataLoader

tokenized_datasets.set_format("torch")
train_dataloader = DataLoader(
    tokenized_datasets["train"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=8,
)
eval_dataloader = DataLoader(
    tokenized_datasets["validation"], collate_fn=data_collator, batch_size=8
)

# **Training the Model**

In [30]:
%pip install sacrebleu \
             evaluate \
             accelerate -q

Note: you may need to restart the kernel to use updated packages.


In [32]:
import evaluate
metric = evaluate.load("sacrebleu")

In [33]:
import numpy as np


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    # In case the model returns more than the prediction logits
    if isinstance(preds, tuple):
        preds = preds[0]

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100s in the labels as we can't decode them
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    return {"bleu": result["score"]}

In [34]:
"""
# Tensorflow not supported
from tensorflow.keras.optimizers.schedules import PolynomialDecay

batch_size = 8
num_epochs = 3

num_train_steps = len(tf_train_dataset) * num_epochs
lr_scheduler = PolynomialDecay(
    initial_learning_rate=5e-5, end_learning_rate=0.0, decay_steps=num_train_steps
)
from tensorflow.keras.optimizers import Adam

opt = Adam(learning_rate=lr_scheduler)
model.compile(optimizer=opt)

# Train in mixed-precision float16
tf.keras.mixed_precision.set_global_policy("mixed_float16")
"""

'\n# Tensorflow not supported\nfrom tensorflow.keras.optimizers.schedules import PolynomialDecay\n\nbatch_size = 8\nnum_epochs = 3\n\nnum_train_steps = len(tf_train_dataset) * num_epochs\nlr_scheduler = PolynomialDecay(\n    initial_learning_rate=5e-5, end_learning_rate=0.0, decay_steps=num_train_steps\n)\nfrom tensorflow.keras.optimizers import Adam\n\nopt = Adam(learning_rate=lr_scheduler)\nmodel.compile(optimizer=opt)\n\n# Train in mixed-precision float16\ntf.keras.mixed_precision.set_global_policy("mixed_float16")\n'

In [35]:
from transformers import AdamW

optimizer = AdamW(model.parameters(), lr=2e-5)

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [36]:
from accelerate import Accelerator

accelerator = Accelerator()
model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader
)

In [37]:
from transformers import get_scheduler

num_train_epochs = 3
num_update_steps_per_epoch = len(train_dataloader)
num_training_steps = num_train_epochs * num_update_steps_per_epoch

lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

In [38]:
def postprocess(predictions, labels):
    predictions = predictions.cpu().numpy()
    labels = labels.cpu().numpy()

    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]
    return decoded_preds, decoded_labels

In [ ]:
from tqdm.auto import tqdm
import torch

progress_bar = tqdm(range(num_training_steps))

for epoch in range(num_train_epochs):
    # Training
    model.train()
    for batch in train_dataloader:
        outputs = model(**batch)
        loss = outputs.loss
        accelerator.backward(loss)

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

    # Evaluation
    model.eval()
    for batch in tqdm(eval_dataloader):
        with torch.no_grad():
            generated_tokens = accelerator.unwrap_model(model).generate(
                batch["input_ids"],
                attention_mask=batch["attention_mask"],
                max_length=128,
            )
        labels = batch["labels"]

        # Necessary to pad predictions and labels for being gathered
        generated_tokens = accelerator.pad_across_processes(
            generated_tokens, dim=1, pad_index=tokenizer.pad_token_id
        )
        labels = accelerator.pad_across_processes(labels, dim=1, pad_index=-100)

        predictions_gathered = accelerator.gather(generated_tokens)
        labels_gathered = accelerator.gather(labels)

        decoded_preds, decoded_labels = postprocess(predictions_gathered, labels_gathered)
        metric.add_batch(predictions=decoded_preds, references=decoded_labels)

    results = metric.compute()
    print(f"epoch {epoch}, BLEU score: {results['score']:.2f}")

    # Save and upload
    accelerator.wait_for_everyone()
    unwrapped_model = accelerator.unwrap_model(model)
    unwrapped_model.save_pretrained('/kaggle/working/out', save_function=accelerator.save)
    if accelerator.is_main_process:
        tokenizer.save_pretrained('/kaggle/working/out')
        repo.push_to_hub(
            commit_message=f"Training in progress epoch {epoch}", blocking=False
        )

  0%|          | 0/18507 [00:00<?, ?it/s]

  0%|          | 0/686 [00:00<?, ?it/s]

# **Evaluation**

In [43]:
my_model= '/kaggle/working/out'
source= 'hin_Deva'
target= 'kan_Deva'

In [44]:
fft_model= AutoModelForSeq2SeqLM.from_pretrained(my_model)

In [47]:
from transformers import pipeline
translator = pipeline('translation', model=fft_model, tokenizer=tokenizer, src_lang=source, tgt_lang=target)

In [63]:
test_dataloader = DataLoader(
    tokenized_datasets["test"], collate_fn=data_collator, batch_size=8
)
test_dataloader

In [60]:
# Acutal translation: हिन्दी-हिमाचली पहाडी काद रोह नाऔ शूनिंदैए दौक मानुष सौर्मींदैसा।
translator("हिन्दी-हिमाचली पहाड़ी भाषा का नाम सुनकर ही कुछ लोग विदक जाते हैं।")

[{'translation_text': 'हिन्दी-हिमाचली पहाड़ी भाषा रोह नाऔ शूनिंदैए दौक मानुष विदक नाशदैसा।'}]

In [61]:
results

{'score': 15.669726436489944,
 'counts': [17796, 7979, 4025, 2089],
 'totals': [46021, 40538, 35080, 29727],
 'precisions': [38.66930314421677,
  19.682766786718634,
  11.473774230330672,
  7.027281595855619],
 'bp': 0.9955770512173886,
 'sys_len': 46021,
 'ref_len': 46225}

In [67]:
test_dataset

Dataset({
    features: ['Hindi', 'Kangri'],
    num_rows: 6093
})

In [75]:
sample_data = test_dataset.select(range(20))  

df = pd.DataFrame(sample_data)

translations = []
for index, row in df.iterrows():
    hindi_text = row['Hindi']
    kangri_target = row['Kangri']

    translation = translator(hindi_text, max_length=128)[0]['translation_text']

    translations.append({
        'Hindi': hindi_text,
        'Target Kangri': kangri_target,
        'Generated Kangri': translation
    })

results_df = pd.DataFrame(translations)
results_df

,Hindi,Target Kangri,Generated Kangri
0,कभी भी किसी कार्य में विघ्न डालने के लिए पंडित...,कौदु बि कुन काम कै मुश्कौल बेज़नाई ताईंयै पंडि...,कौदु बि कुन काम कै विघ्न बेज़नाई ताईंयै पंडित ...
1,हरगंगे! भई हरगंगे! \n,हरगंगे! भाई हरगंगे! \n,हरगंगे! भाई हरगंगे!
2,क्योंकि यह मिट्टी \n,किजु बोलताह होई माटिह\n,किजु बोल्ताह होई माटिह
3,यह जगह शहर में सबसे अच्छा बर्गर है! \n,हौयौ ज़ागा कैह शहर रोह सेबका औबौल बर्गरस। \n,होई ज़ागा शहर केह सऐब काह बोलोह बर्गर साह।
4,कितना बदल गया\n,केतिग बोदली।\n,केतिग बोदलुई
5,नई नई दलीलें अपीलें करके \n,नोऊवोंह-नोऊवोह बाते होरिह ज़ुर्राई देह।\n,नोईयोह नोईयोह दलील अपील टानिह देह
6,उसने झट से सुदामा की भेड़ चुराकर अपनी भेड़ों ...,तियूँवै फौटौक ज़ैई सुदामा रि बेरी च़ोरिंदै आपन...,तिनिये सोदामा रोह बेरी च़ोरिह देह आफरोह बेरी क...
7,हाथ जोड़ आते यहाँ लाखों ही सवाली\n,हाथ जोडाईदेह आछदेह साह ईनचे लाखों सवाली \n,हाथ जोडाईदेह आछ़तोह ईन्छ़े लाखो सवाली
8,या यूं कहें वे नीला आकाश न देख पाने को अभिशप्त...,या हैऔ बोलनौ तेनै नीला आकाश नेदेखी बोलबैलै दोश...,याह होयोह बोलेन थियेह सोह निलोह सोरोग नाई शाईम...
9,कुश्ती में पहने हुए उसके नए कपड़े लगाकर कुछ प...,कुश्ती कै लाउवैंदै तियूँरै नौउवै गाछ़ै गिनुविं...,कुश्ती कैह बेज़िह देह तिन रोह नोईयोह गाछ़ोह बे...
